<a href="https://colab.research.google.com/github/AndreyKuratov/project_mldm_21/blob/main/MLDM_proj_4_simple_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузка библиотек

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf
import scipy.signal as signal
import os
from tqdm import tqdm

import os
import csv
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image

import librosa as lb 
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

from skimage.transform import resize
from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from skimage.transform import resize

# 1. Загрузка данных и меток

In [14]:
PATH_TO_DIR = '/content/drive/MyDrive/mldm21_project/'
PATH_TO_SAVE = '/content/drive/MyDrive/mldm21_project/'

data = np.load(PATH_TO_SAVE+'sr_48e3_nfft_2e11_hlop_2e9_pict_256x512_u8.npz',allow_pickle=True)
# 'imgs' 'labels'
labels = np.load(PATH_TO_SAVE+'labels_simple.npz')
# 'files_fp' 'labels_fp' 'files_tp' 'labels_tp'

labels_pd_fp = pd.read_csv('/content/drive/MyDrive/mldm21_project/train_fp.csv')
labels_pd_tp = pd.read_csv('/content/drive/MyDrive/mldm21_project/train_tp.csv')


пордок метод одиаков:

In [22]:
np.sum(data['labels']!=labels['files_tp'])

0

Выбор меток которые соотвествуют TP с одним полезным сигналом, а так же преобразование выбор матриц-рисунков для этих меток

In [23]:
lbl,cnt_lbl = np.unique(labels_pd_tp['recording_id'].values,return_counts=True)

In [28]:
lbl_single = lbl[cnt_lbl==1]
length_1tp = len(lbl_single)
print(length_1tp)


1065


Извлечение данных

In [48]:
mask = np.isin(data['labels'], lbl_single)

In [50]:
data_to_nn = data['imgs'][mask].copy()
labels_to_nn = labels['labels_tp'][mask].copy()

In [52]:
np.savez(PATH_TO_SAVE+'1signal_tp',data=data_to_nn,labels=labels_to_nn)

# Исползование моделей

In [75]:
import torchvision.models as models
import torch

import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

#resnet18 = models.resnet18()
#alexnet = models.alexnet()
#vgg16 = models.vgg16()
#squeezenet = models.squeezenet1_0()
#densenet = models.densenet161()
#inception = models.inception_v3()
#googlenet = models.googlenet()
#shufflenet = models.shufflenet_v2_x1_0()
#mobilenet_v2 = models.mobilenet_v2()
#mobilenet_v3_large = models.mobilenet_v3_large()
#mobilenet_v3_small = models.mobilenet_v3_small()
#resnext50_32x4d = models.resnext50_32x4d()

In [82]:
NUM_CLASSES = 3


num_classes = NUM_CLASSES


*   **resnet18,34,50**
*   **vgg16**
*   **alexnet**
*   **squeezenet**
*   **inception**
*  **mobilenet_v3_small**



In [139]:
resnet18 = models.resnet18()
resnet18.fc = nn.Linear(512, num_classes)

resnet34 = models.resnet34()
resnet34.fc = nn.Linear(512, num_classes)

resnet50 = models.resnet50()
resnet50.fc = nn.Linear(2048, num_classes)


alexnet = models.alexnet()
alexnet.classifier[6] = nn.Linear(4096,num_classes)

vgg16 = models.vgg16()
vgg16.classifier[6] = nn.Linear(4096,num_classes)

squeezenet = models.squeezenet1_0()
squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))

inception = models.inception_v3(aux_logits=False)  #'N x 3 x 299 x 299'
#inception.AuxLogits.fc = nn.Linear(768, num_classes)
inception.fc = nn.Linear(2048, num_classes)

mobilenet_v3_small = models.mobilenet_v3_small()
mobilenet_v3_small.classifier[0] = nn.Linear(in_features=576, out_features=4096, bias=True)
mobilenet_v3_small.classifier[3] = nn.Linear(4096,num_classes,bias=True)



/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


In [140]:
model_stack = [resnet18,resnet34,resnet50, alexnet, vgg16, squeezenet,inception, mobilenet_v3_small ]
ModelNames = [str(imodel).split('(')[0] for imodel in model_stack]

Пример батча

In [141]:
init = torch.Tensor(data_to_nn[0:4]).transpose(1,3)

for imodel in [resnet18,resnet34,resnet50,alexnet, vgg16, squeezenet,mobilenet_v3_small ]:
  ModelName = str(imodel).split('(')[0]
  print(ModelName) 
  print(imodel(init).size(),'\n')

ResNet
torch.Size([4, 3]) 

ResNet
torch.Size([4, 3]) 

ResNet
torch.Size([4, 3]) 

AlexNet
torch.Size([4, 3]) 

VGG
torch.Size([4, 3]) 

SqueezeNet
torch.Size([4, 3]) 

MobileNetV3
torch.Size([4, 3]) 



In [144]:
from skimage.transform import resize
len_N = 4
init_incept = torch.Tensor(resize(data_to_nn[0:len_N],(len_N,299,299,3))).transpose(1,3)
init_incept.size()


torch.Size([4, 3, 299, 299])

In [143]:
output = inception(init_incept)
torch.nn.functional.softmax(output, dim=0)


tensor([[0.1990, 0.3022, 0.3035],
        [0.3283, 0.3180, 0.2667],
        [0.2399, 0.1649, 0.2031],
        [0.2327, 0.2148, 0.2267]], grad_fn=<SoftmaxBackward0>)